# The Fastai `GetAttr` class 
> And some examples. As a summary, the `GetAttr` class passes `getattr` lookup to `cls._default` where `cls` inherits from `GetAttr`.
- toc: true
- badges: true
- comments: true
- author: Aman Arora

Let's check the `GetAttr` source code:

In [2]:
class GetAttr:
    "Inherit from this to have all attr accesses in `self._xtra` passed down to `self.default`"
    _default='default'
    def _component_attr_filter(self,k):
        if k.startswith('__') or k in ('_xtra',self._default): return False
        xtra = getattr(self,'_xtra',None)
        return xtra is None or k in xtra
    def _dir(self): return [k for k in dir(getattr(self,self._default)) if self._component_attr_filter(k)]
    def __getattr__(self,k):
        if self._component_attr_filter(k):
            attr = getattr(self,self._default,None)
            if attr is not None: return getattr(attr,k)
        raise AttributeError(k)
    def __dir__(self): return custom_dir(self,self._dir())
#     def __getstate__(self): return self.__dict__
    def __setstate__(self,data): self.__dict__.update(data)

The `_component_attr_filter` checks if the attribute being looked for is not starting with '__' (dunder thingies) or called '_xtra' or `self._default`. If it's either of these, return value is False. 

Otherwise, if there is some attribute `_xtra` defined inside the class, then the return is `True` if `k` exists in `_xtra`. OR, return is also `True` if `xtra` is None. 

Essentially, a call to `_component_attr_filter` returns True if `self._xtra` is None or `k` is part of `self._xtra`. It returns `False` if the attribute being looked for is one of dunder thingies or one of `self._xtra` or `self._default`.

Finally, when a call to `__getattr__` is made, first we check for the attribute `self._default` if `_component_attr_filter` returns `True`. Next, we look for the attribute `k` being looked inside the `self._default` attribute thus, passing the `getattr` to the class's own attr `self._default`. 

Let's see an example:

In [10]:
class Robot:
    name = "IRobot"
    
class Hand(GetAttr):
    _default = "robot"
    def __init__(self): self.robot = Robot

In [11]:
hand = Hand()
hand.name

'IRobot'

Even the attribute `name` didn't exist in `Hand` class, we still get the value 'IRobot' because this class inherits from `GetAttr` and passes the attribute lookup down to `self._default` class which in this case is `self.robot`.

One of the classes that inherits `GetAttr` is `DataLoader` and `_default` is set to `dataset` which generally is `DataSets`. 

In [12]:
path = untar_data(URLs.MNIST_TINY)
dset = Datasets(get_image_files(path))
dl = DataLoader(dset)

Now let's have a look at `dataloaders` method. This returns a `bound method FilteredBase` which actually `DataSets` inherits from. 

As can be seen from the `MRO` below: 

In [15]:
Datasets.__mro__

(fastai2.data.core.Datasets, fastai2.data.core.FilteredBase, object)

But, this method is made available to `DataLoader` because `_default` was set to `DataSets` and this is due to the magic of `GetAttr`. 

In [14]:
dl.dataloaders

<bound method FilteredBase.dataloaders of (#1428) [(Path('/home/ubuntu/.fastai/data/mnist_tiny/valid/7/7565.png'),),(Path('/home/ubuntu/.fastai/data/mnist_tiny/valid/7/7445.png'),),(Path('/home/ubuntu/.fastai/data/mnist_tiny/valid/7/9169.png'),),(Path('/home/ubuntu/.fastai/data/mnist_tiny/valid/7/7924.png'),),(Path('/home/ubuntu/.fastai/data/mnist_tiny/valid/7/7197.png'),),(Path('/home/ubuntu/.fastai/data/mnist_tiny/valid/7/7300.png'),),(Path('/home/ubuntu/.fastai/data/mnist_tiny/valid/7/9024.png'),),(Path('/home/ubuntu/.fastai/data/mnist_tiny/valid/7/9816.png'),),(Path('/home/ubuntu/.fastai/data/mnist_tiny/valid/7/7180.png'),),(Path('/home/ubuntu/.fastai/data/mnist_tiny/valid/7/9515.png'),)...]>